<a href="https://colab.research.google.com/github/bellabf/RagDerby/blob/main/RagDerby.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Breaking Language Barriers in Sports: A Multilingual Question-Answering System for Roller Derby

This notebook has been contributed by **[Isabella Bicalho Frazeto](https://www.linkedin.com/in/isabella-frazeto/)** from the *Cohere For AI Community* 🏆

## Motivation

Remember playing in the schoolyard?

Background and speakign different languages didn't matter, and somehow everybody played coordinating complex games with ever-changing rules. A stick becomes a hockey stick, a wall becomes a goal, and hand gestures and smiles bridge any language gap.

We didn't need rulebooks then – play was universal. But as adults, our relationship with sports changes.

In sports, we often find **structure, safety, and community**. For mainstream sports like football or basketball, the decades of cultural integration have created a shared universal language of play.

![handwritten  project overview](https://github.com/bellabf/RagDerby/blob/main/images/1.png?raw=true)

But for emerging sports, culturally specific activities, or underrepresented athletics like roller derby, korfball, or sepak takraw, that transition isn't so simple. Language barriers become more than just communication hurdles – they become gatekeepers to participation, safety, and community building.

### Breaking Language Barriers in Sports: Why Roller Derby?

Picture this: A fast-paced sport on wheels, played across six continents, from Buenos Aires to Bangkok, Tokyo to Toronto, Moscow to Melbourne. Roller derby has evolved from its American roots into a truly global phenomenon, with over 4,000 leagues worldwide. Yet, there's a catch – the official rulebook, the very foundation of the sport, is only available in four languages: English, Spanish, French, and German.

### Why not just use Google Translate?


Well, **Context is Everything**
   - When a player reads a machine-translated rule about a "jam," are they thinking about fruit preserves or a two-minute scoring opportunity?
   - Will the skater understand that a "pack" isn't something you carry on your back, but a critical group of players that defines legal gameplay?
   - What about the translation of "recycling" in derby (that has nothing to do with environmental conservation)?


## Multilingual question-answering system

We need something that can handled that context and make things more digestable for the players. We can create a tool that doesn't just translate words, but translates understanding. Our goal is to make roller derby truly accessible to everyone, regardless of their native language.


## Project Overview

This notebook demonstrates how to build a multilingual question-answering system for sports rules that are under represented. We use the WFTDA (Women's Flat Track Derby Assiociation) rules and gameplay. Our goal is to make this exciting sport more accessible to people worldwide by breaking down language barriers. We will advanced language models and RAG (Retrieval Augmented Generation) technology to provide accurate information about roller derby in multiple languages.

We will combine langchain with the brande new `c4ai-aya-23-35b` or MysteryBot to do so.

![mystery bot and langchain being friends](https://github.com/bellabf/RagDerby/blob/main/images/4.png?raw=true)





## Technical Implementation Steps

### 1. Setting Up Dependencies
First, we install and import all necessary libraries. Our system relies on:
- `faiss-cpu` for efficient similarity search
- `langchain`, `langchain_community`, `langchain_core` components for building our pipeline
- `cohere` for language processing capabilities

In [1]:
!pip install numpy faiss-cpu cohere langchain langchain-core langchain-cohere pydantic 

### Setting up the imports

In [2]:
import os
import cohere
import faiss
import numpy as np

from typing import Any, List, Optional
from pprint import pprint
from getpass import getpass
from operator import itemgetter

from pydantic import Field, PrivateAttr

from langchain.llms.base import LLM
from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.prompts import PromptTemplate, FewShotPromptTemplate
from langchain.schema.runnable import RunnableParallel, RunnablePassthrough
from langchain.chains import RetrievalQA
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever

from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain_cohere.llms import Cohere
from langchain_cohere import CohereRerank

from langchain_core.retrievers import BaseRetriever
from langchain_core.documents import Document
from langchain_core.callbacks.manager import CallbackManagerForRetrieverRun


### 2. API Configuration
We set up our connection to Cohere's API. This gives us access to powerful language models that will handle our multilingual processing needs.


In [3]:
api_key = getpass("Please enter your Cohere API key: ")

os.environ["COHERE_API_KEY"] = api_key

### 3. Custom Implementations
We will create a couple of custom elements:


*   A custom wrapper for Cohere's API `CohereLLM`
*   A Faiss retriever `FAISSRetriever`
*   Load documents `load_documents`

One of these things are not like the others 🤔

In [4]:
# Custom wrapper
class CohereLLM(LLM):
    """Custom LLM wrapper for Cohere's API."""

    model_name: str = Field(..., description="The name of the Cohere model to use")
    temperature: float = Field(default=0.7, description="Sampling temperature")
    max_tokens: int = Field(
        default=256, description="Maximum number of tokens to generate"
    )

    _client: Any = PrivateAttr()

    def __init__(self, api_key: str, **kwargs):
        """Initialize the Cohere client."""
        super().__init__(**kwargs)
        self._client = cohere.Client(api_key)

    @property
    def _llm_type(self) -> str:
        """Return type of LLM."""
        return "cohere_custom"

    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:
        """Execute the text generation with Cohere."""
        try:
            response = self._client.generate(
                prompt=prompt,
                model=self.model_name,
                temperature=self.temperature,
                max_tokens=self.max_tokens,
            )
            # Get the first generated text
            if response.generations:
                return response.generations[0].text
            return ""
        except Exception as e:
            raise ValueError(f"Error calling Cohere API: {str(e)}")


# Create a retriever class
class FAISSRetriever(BaseRetriever):
    faiss_index: Any = Field(description="FAISS index for similarity search")
    stored_texts: List[Document] = Field(description="List of stored documents")
    client: Any = Field(description="Cohere client instance")

    @classmethod
    def from_params(cls, index: Any, texts: List[Document], client: Any) -> "FAISSRetriever":
        return cls(
            faiss_index=index,
            stored_texts=texts,
            client=client
        )

    def _get_relevant_documents(
        self, query: str, *, run_manager: CallbackManagerForRetrieverRun
    ) -> List[Document]:
        query_embedding = self.client.embed(
            texts=[query],
            model='embed-english-v3.0',
            input_type='search_query'
        ).embeddings[0]

        _, indices = self.faiss_index.search(
            np.array([query_embedding]).astype('float32'),
            6  # k=6
        )
        return [self.stored_texts[i] for i in indices[0]]

def load_document(file_path: str) -> Document:
    """Load a text file and return a Document object."""
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            text = file.read()
        return Document(page_content=text)
    except UnicodeDecodeError:
        # Fallback to latin-1 if UTF-8 fails
        with open(file_path, 'r', encoding='latin-1') as file:
            text = file.read()
        return Document(page_content=text)



##### Here is a base overview of how to use `CohereLLM`

In [5]:
# Declare our model
llm = CohereLLM(
    api_key=api_key, model_name="mystery-model", temperature=0.8, max_tokens=1000
)

# Prompt template
prompt = PromptTemplate(input_variables=["task"], template="{task}")

# Basic chain
chain = prompt | llm
response = chain.invoke(input="Give two reasons why sports bring people together!")
pprint(response)

('Sports have a unique ability to bring people together in numerous ways. Here '
 'are two significant reasons:\n'
 '\n'
 '1. **Common Interest and Identity**: Sports provide a shared interest and a '
 "sense of identity for people from diverse backgrounds. Whether it's "
 'supporting a local team, participating in a sport, or simply enjoying a '
 'game, sports fans and athletes often find themselves connected through a '
 'common passion. This shared interest transcends social, cultural, and '
 'economic barriers, fostering a sense of community and unity. For example, '
 'people from different neighborhoods or countries can come together to cheer '
 'for their favorite team, creating a bond based on their collective love for '
 'the sport.\n'
 '\n'
 '2. **Teamwork and Collaboration**: Many sports are team-based, which '
 'encourages collaboration, communication, and cooperation. Playing or '
 'coaching a team sport requires individuals to work together towards a common '
 'goal, such 

##### Basics check ✅

Once we have the basic syntax, we can move on to creating the chains we will use for the multilingual q&a system. We will use five chains in this project.

![chains](https://github.com/bellabf/RagDerby/blob/main/images/2.png?raw=true)

### 4. Language Detection System
We implement a language detection system that can automatically identify the input language of user queries. We do this to determine when a translation is needed.

In [6]:
language_detection_prompt = PromptTemplate(
    input_variables=["text"],
    template="""
    Given the following text, identify the language it is written in:

    Text: "{text}"

    Please respond with the name of the language only.
    """,
)

chain_detect = language_detection_prompt | llm
response = chain_detect.invoke(input="Hoe lang blijft een speler in de penaultybox?")
pprint(response)

'Dutch'



### 5. Translating user queries from any supported language to English

For both translating chains, we will use Few-Shot Prompting.

In [7]:
examples = [
    {"question": "Qual o papel da pivot?", "answer": "What is the pivot's role?"},
    {
        "question": "Come si gioca a roller derby?",
        "answer": "How do you play roller derby?",
    },
    {"question": "Qui est la jammeneuse?", "answer": "Who is the jammer?"},
]

example_template = """
Question: {question}
English: {answer}"""

example_prompt_to_en = PromptTemplate(
    input_variables=["question", "answer"], template=example_template
)

# Define the few-shot prompt
few_shot_prompt_to_en = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt_to_en,
    prefix="Here are some examples of English translations:",
    suffix="\nQuestion: {question}\nEnglish:",
    input_variables=["question"],
)


chain_to_en = few_shot_prompt_to_en | llm

response = chain_to_en.invoke(input={"question": "Jak długo trwa mecz?"})  # polish
pprint(response)

'How long does a match last?'



### 6. Building the RAG System

The Retrieval Augmented Generation system combines:
- Document loading and splitting for roller derby content
- FAISS vector store
- Contextual compression
- Question-answering
Adding contextual compression reduce hallucination in our early experiment setup.

[If you cannot find the output.txt, please redownload it in here](https://github.com/bellabf/RagDerby/blob/main/data/output.txt)

In [8]:
# Load and split document
doc = load_document("data/output.txt")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100, add_start_index=True)
texts = text_splitter.split_documents([doc])

# Initialize Cohere c
co = cohere.Client(api_key)

# Create base retriever using FAISS
embeddings = []
raw_texts = [doc.page_content for doc in texts]
response = co.embed(texts=raw_texts, model='embed-english-v3.0', input_type='search_document')
embeddings = response.embeddings

# Initialize FAISS
embedding_dim = len(embeddings[0])
index = faiss.IndexFlatL2(embedding_dim)
index.add(np.array(embeddings).astype('float32'))

# Create the base retriever
base_retriever = FAISSRetriever.from_params(index, texts, co)

# Create compression retriever using ContextualCompressionRetriever
compressor = CohereRerank(model="rerank-english-v3.0")
compression_retriever = ContextualCompressionRetriever(
    base_retriever=base_retriever,
    base_compressor=compressor
)

# Question-answering chain
chain_rag = RetrievalQA.from_chain_type(
    llm=Cohere(temperature=0),
    retriever=compression_retriever
)


In [9]:
results = chain_rag.invoke({"query": "How long does roller derby last?"})
pprint(results["result"])

(' A complete roller derby game consists of two 30-minute periods with a '
 'halftime between them and lasts for 60 minutes of play. The periods are '
 'broken into 2 minute jams that may be called off prior to two minutes. At '
 'least 30 seconds must elapse between jams. ')



### 5.Translating responses from English back to the user's original language

We do this to ensure accurate information retrieval while maintaining natural dialogue in the user's preferred language.

In [10]:
examples = [
    {
        "input": "Qual o papel da pivot?",
        "language": "Portuguese",
        "translation": "What is the pivot's role?",
    },
    {
        "input": "Come si gioca a roller derby?",
        "language": "Italian",
        "translation": "How do you play roller derby?",
    },
    {
        "input": "What is a team timeout?",
        "language": "English",
        "translation": "What is a team timeout?",
    },
    {
        "input": "How long does a penalty last?",
        "language": "English",
        "translation": "How long does a penalty last?",
    },
]

# Template for each example
example_template = """
Input: {input}
Language: {language}
Translation: {translation}"""

example_prompt_to_target = PromptTemplate(
    input_variables=["input", "language", "translation"], template=example_template
)

few_shot_prompt_to_target = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt_to_target,
    prefix="""You are a language translator. Provide the translation of the input only. Here are some examples:""",
    suffix="""
Input: {input_text}
Language: {target_language}
Translation:""",
    input_variables=["input_text", "target_language"],
)


chain_to_target = few_shot_prompt_to_target | llm

response = chain_to_target.invoke(
    input={
        "input_text": "How do I indentify the referees?",
        "target_language": "portuguese",
    }
)
pprint(response)

'Como eu identifico os árbitros?'


##### Chains ✅

Now, we just need to put them together.

#### Let's understand step by step

The ouput of one chain is the input of the next.
Let'see the example below.

In [11]:
chain_to_en.invoke("Como é a pista de roller derby?")

'How is the roller derby track?\n\nLet\'s break down the translations:\n\n1. **Qual o papel da pivot?**\n   - English: What is the pivot\'s role?\n   - Here, "pivot" refers to a specific position or player in roller derby.\n\n2. **Come si gioca a roller derby?**\n   - English: How do you play roller derby?\n   - This translates the question about the rules or method of playing roller derby.\n\n3. **Qui est la jammeneuse?**\n   - English: Who is the jammer?\n   - "Jammeneuse" is a French term adopted into roller derby to refer to the "jammer" position, similar to the English term.\n\n4. **Como é a pista de roller derby?**\n   - English: How is the roller derby track?\n   - This asks about the layout, design, or features of the roller derby track.'

In [12]:
chain_rag(chain_to_en.invoke("Como é a pista de roller derby?"))

/tmp/ipykernel_342690/1091893329.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain_rag(chain_to_en.invoke("Como é a pista de roller derby?"))


{'query': 'How is the roller derby track?\n\nLet\'s break down the translations:\n\n1. **Qual o papel da pivot?**\n   - English: What is the pivot\'s role?\n   - Here, "pivot" refers to a specific position in roller derby, similar to how it\'s used in other sports (e.g., basketball).\n\n2. **Come si gioca a roller derby?**\n   - English: How do you play roller derby?\n   - This translates the question about the rules or general gameplay of roller derby.\n\n3. **Qui est la jammeneuse?**\n   - English: Who is the jammer?\n   - In roller derby, a "jammer" is a player with a specific role. This question asks for the identity of that player.\n\n4. **Como é a pista de roller derby?**\n   - English: How is the roller derby track?\n   - This inquires about the design, layout, or features of the roller derby track.',
 'result': ' The track is flat and oval in shape. There are two jammer lines, one at each end of the track, and a pivot line at the center. The rollers skate around the track in a 


### 7. Multi-Step Processing Pipeline


Our final pipeline integrates all components:
1. Detect input language
2. Translate query to English if needed
3. Retrieve relevant roller derby information
4. Translate response back to original language
5. Deliver natural, accurate responses


In [13]:
# Create the pipeline using your existing chains
derby_qa_pipeline = (
    # First detect language
    RunnableParallel(
        original_query=RunnablePassthrough(),
        detected_language=lambda x: chain_detect.invoke({"text": x}),
    )
    |
    # Then handle translation to English
    RunnableParallel(
        original_query=itemgetter("original_query"),
        detected_language=itemgetter("detected_language"),
        translated_query=lambda x: (
            chain_to_en.invoke({"question": x["original_query"]})
            if x["detected_language"].lower().strip() != "english"
            else x["original_query"]
        ),
    )
    |
    # Use RAG to get answer in English
    RunnableParallel(
        query=itemgetter("translated_query"),
        answer=lambda x: chain_rag.invoke({"query": x["translated_query"]}),
        original_query=itemgetter("original_query"),
        detected_language=itemgetter("detected_language"),
    )
    |
    # Translate answer back to original language
    RunnableParallel(
        original_query=itemgetter("original_query"),
        detected_language=itemgetter("detected_language"),
        final_answer=lambda x: (
            chain_to_target.invoke(
                {"input_text": x["answer"], "target_language": x["detected_language"]}
            )
            if x["detected_language"].lower().strip() != "english"
            else x["answer"]
        ),
    )
)


### 8. Example Queries and Testing

We will the functions `process_derby_query` to helps see what the pipeline is doing.

In [14]:
def process_derby_query(query: str):
    try:
        result = derby_qa_pipeline.invoke(query)

        print("\nQuery Processing Steps:")
        print(f"1. Original Query: {result['original_query']}")
        print(f"2. Detected Language: {result['detected_language']}")
        print(
            f"3. Final Answer in {result['detected_language']}: {result['final_answer']}"
        )

    except Exception as e:
        print(f"Error in pipeline: {str(e)}")


In [15]:
process_derby_query("What is roller derby?")


Query Processing Steps:
1. Original Query: What is roller derby?
2. Detected Language: English
3. Final Answer in English: {'query': 'What is roller derby?', 'result': " Roller derby is a full-contact sport played on a flat oval track, involving two teams of five players where points are scored by lapping members of the opposing team. \nPlayers wear roller skates and cannot use their heads, elbows, forearms, hands, knees, lower legs, or feet to make contact with opponents. \n\nI hope this information is helpful! Let me know if there's anything else you'd like to know. "}


In [16]:
process_derby_query("Como são marcados pontos?")


Query Processing Steps:
1. Original Query: Como são marcados pontos?
2. Detected Language: Portuguese
3. Final Answer in Portuguese: {'query': 'Como os pontos são marcados?', 'result': 'Os Jammers marcam um ponto toda vez que passam um Blocker adversário duas vezes seguidas. \nSe um oponente for passado, mas um ponto não for marcado (porque a contagem de passes não foi atingida), o Jammer pode ceder a posição para esse oponente e repassá-lo, ganhando um passe, para marcar nesse oponente. \nDessa forma, os Jammers podem marcar em cada oponente durante cada passagem pelo grupo. '}


In [17]:
process_derby_query("comment identifier la jammeuse?")


Query Processing Steps:
1. Original Query: comment identifier la jammeuse?
2. Detected Language: French
3. Final Answer in French: {'query': 'Comment identifie-t-on le jammer ?', 'result': 'Le jammer est le patineur qui porte le couvre-casque de jammer, ou la star, au début de la manche. Le jammer peut également être identifié comme le patineur qui a été désigné comme tel lors de la manche précédente. \n\nSi aucun patineur n'est sur la piste avec la star ou si aucun patineur n'a été désigné lors de la manche précédente, la manche ne commencera pas et des pénalités peuvent être appliquées.'}


In [18]:
process_derby_query("Explain rollerderby to me like I am five")


Query Processing Steps:
1. Original Query: Explain rollerderby to me like I am five
2. Detected Language: English
3. Final Answer in English: {'query': 'Explain rollerderby to me like I am five', 'result': " Roller derby is a fun, full-contact sport played by two teams on a flat oval track. It's like a race, but with more action and players. Each team has five players on the track at once, and they're divided into two groups: blockers and jammer. The jammer wears a special helmet cover with a star and scores points by racing past the other team's blockers. It's like they're weaving through obstacles on the track! But there are some rules to keep everyone safe: players can't use their heads or certain body parts to push or tackle each other. It's important to be respectful and not use bad words towards teammates or officials. So it's a fast-paced competitive sport where players score points by skating fast and smart! "}


#### Pipeline ✅

We can also make queries directly to the pipeline

In [19]:
print("Query 1:")
pprint(derby_qa_pipeline.invoke("What is legal contact area to impact in roller derby?")['final_answer']['result'])
print("Query 2:")
pprint(derby_qa_pipeline.invoke("What is legal contact zone to impact in roller derby?")['final_answer']['result'])

Query 1:
(' The sport of roller derby has designated legal target zones in section '
 '4.1.1. These legal target zones include:\n'
 '\n'
 '- Arms, front and side\n'
 '- Front of body, above the waist\n'
 '- Front of thighs\n'
 '\n'
 'These are the designated areas in roller derby where players are allowed to '
 'make contact with other players through blocking or other actions. ')
Query 2:
(' Impact to an Illegal Target Zone should be penalized based on the impact it '
 'has on the target. Illegal Target Zones include:\n'
 '\n'
 'Back of the body, including the back of the buttocks and the back of the '
 'thighs\n'
 'Head, down to the collarbone\n'
 'Below mid-thigh\n'
 '\n'
 'For safety reasons, avoidable forceful contact to the back, or any forceful '
 'contact to the head or neck should be penalized regardless of impact.\n'
 '\n'
 'A Skater suddenly presenting an Illegal Target Zone to an opponent, giving '
 'that opponent no reasonable opportunity to avoid illegal contact, is '
 'c

We can see that slightly different queries still give us good results

In [20]:
pprint(derby_qa_pipeline.invoke("What are the penalties in derby?")['final_answer']['result'])

(' The penalties in roller derby take many different forms but are ultimately '
 'punishments for illegal actions that impact the game. \n'
 'These penalties can be assessed to skaters as well as anyone else involved '
 'in the game such as team staff, officials, mascots, event staff, and '
 'spectators. \n'
 '\n'
 'The Rules of Flat Track Roller Derby Casebook goes into more specific detail '
 'about the types of penalties that can be assessed for unsporting conduct, as '
 'well as guidelines for referees to make judgments about these penalties. ')


In [21]:
pprint(derby_qa_pipeline.invoke("Quando um star pass é valido?")['final_answer'])

("{'query': 'Quando uma star pass é válida?', 'result': 'Uma star pass é "
 'válida quando o jammer entrega a star ao seu pivot enquanto ambos os '
 'patinadores estão eretos, dentro do limite, em jogo e nenhum patinador está '
 'indo para a caixa de penalidade. \\nIsso garante que a transferência de '
 'posse seja conduzida de maneira justa e não cause perturbação à equipe '
 "adversária.'}")


# And that is it!


![globe in skates in a roller derby track](https://github.com/bellabf/RagDerby/blob/main/images/3.png?raw=true)

